In [10]:
import os
import csv
from sklearn.utils import shuffle
from scipy.ndimage import rotate
import cv2
import numpy as np
import sklearn
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Reshape
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.backend import tf as ktf
import matplotlib.image as mpimg

#Read csv file to get all data
samples = []
with open('./img10/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

#split trainging samples and validation samples
train_samples, validation_samples = train_test_split(samples, test_size=0.2)
train_samples = sklearn.utils.shuffle(train_samples)
validation_samples = sklearn.utils.shuffle(validation_samples)
iglobal=0 # use this for debugging to see how many time generator yields
def generator(samples, batch_size=32, validation_flag=True):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            global iglobal
            iglobal=iglobal+1
            #debug to see how many times yields
            #print ("yield", iglobal, "times")
            for batch_sample in batch_samples:
                for i in range(3):
                    name = './img10/IMG/'+batch_sample[i].split('/')[-1]
                    #image = cv2.imread(name)
                    image=mpimg.imread(name)
                    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #image = exposure.equalize_hist(image)
                    angle = float(batch_sample[3])
                    if i==1: #left camera images 
                        angle=angle+0.23
                    if i==2: #right camera images
                        angle=angle-0.23
                        
                    
                    images.append(image)
                    angles.append(angle)
                    
                    if i==9: # this won't be executed as I find the flip image will add lots of noises
                        images.append(np.fliplr(image))
                        angles.append(angle*-1)
                    
                    if i==9: #this won't be exucuted das I can't find proper angle adjustment for rorate images
                        Rotate_image = rotate(image, 5, reshape=False) 
                        images.append(Rotate_image)
                        angles.append(angle-0.2)
                        Rotate_image = rotate(image, -5, reshape=False) 
                        images.append(Rotate_image)
                        angles.append(angle+0.2)
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function, select batch size with 16. If set it with 32, sometimes will encounter OOM exception
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)
print(len(train_samples))
print(len(validation_samples))
def resize_image(image):
    # In Nvidia CNN, the image input size is 66 x 200
    from keras.backend import tf as ktf   
    resized_image = ktf.image.resize_images(image, (66, 200))
    return resized_image

model=Sequential()
#nomalized images
model.add(Lambda(lambda x: x /255 - 0.5, input_shape=(160,320,3)))
#Cropping the image to only see the middle part and remove the noise
model.add(Cropping2D(cropping=((70,25), (0,0))))
#REsize the image to Nvidia input size
model.add(Lambda(resize_image, input_shape=(65,320,3), output_shape=(66, 200, 3)))

#Nvidia 5 lays of CNN with dropout 0.25 on each layer
model.add(Convolution2D(24,5,5,border_mode='valid', activation='relu', subsample=(2,2)))
model.add(Dropout(0.25))
model.add(Convolution2D(36,5,5,border_mode='valid', activation='relu', subsample=(2,2)))
model.add(Dropout(0.25))
model.add(Convolution2D(48,5,5,border_mode='valid', activation='relu', subsample=(2,2)))
model.add(Dropout(0.25))
model.add(Convolution2D(64,3,3,border_mode='valid', activation='relu', subsample=(1,1)))
model.add(Dropout(0.25))
model.add(Convolution2D(64,3,3,border_mode='valid', activation='relu', subsample=(1,1)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(1164, activation='relu'))


model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

print(model.summary())

#use adam optimizer
model.compile(loss='mse', optimizer='adam')


#stop early to prevent overfitting, the tolerance is 2 epoches
filepath="model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# check 2 epochs patience
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1, mode='min') 

callbacks_list = [checkpoint, early_stop]

#I set samples_per_epoch as len(train_samples)/batch_size +1 as I think for each epoch, looping through the whole training set is enough
model.fit_generator(train_generator, samples_per_epoch= len(train_samples)/32+1, validation_data=validation_generator, nb_val_samples=len(validation_samples)/32+1, nb_epoch=100,callbacks=callbacks_list, verbose=1)
#model.save('model.h5')
print ("model was saved successfully")

8428
2107


D:\ProgramData\Anaconda3\envs\3point6\lib\site-packages\ipykernel_launcher.py:95: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2), padding="valid")`
D:\ProgramData\Anaconda3\envs\3point6\lib\site-packages\ipykernel_launcher.py:97: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2), padding="valid")`
D:\ProgramData\Anaconda3\envs\3point6\lib\site-packages\ipykernel_launcher.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2), padding="valid")`
D:\ProgramData\Anaconda3\envs\3point6\lib\site-packages\ipykernel_launcher.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", strides=(1, 1), padding="valid")`
D:\ProgramData\Anaconda3\envs\3point6\lib\site-packages\ipykernel_launcher.py:103: UserWarning: Update your `Conv2D` call to the Keras 2 API: `

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_15 (Lambda)           (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_8 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
lambda_16 (Lambda)           (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 31, 98, 24)        1824      
_________________________________________________________________
dropout_36 (Dropout)         (None, 31, 98, 24)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 14, 47, 36)        21636     
_________________________________________________________________
dropout_37 (Dropout)         (None, 14, 47, 36)        0         
__________

D:\ProgramData\Anaconda3\envs\3point6\lib\site-packages\ipykernel_launcher.py:132: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
D:\ProgramData\Anaconda3\envs\3point6\lib\site-packages\ipykernel_launcher.py:132: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., callbacks=[<keras.ca..., verbose=1, steps_per_epoch=264.375, epochs=100, validation_steps=66.84375)`


Epoch 1/100
265/264 [==============================] - 85s 320ms/step - loss: 0.0327 - val_loss: 0.0288
Epoch 2/100
265/264 [==============================] - 83s 312ms/step - loss: 0.0280 - val_loss: 0.0291
Epoch 3/100
265/264 [==============================] - 83s 312ms/step - loss: 0.0272 - val_loss: 0.0281
Epoch 4/100
265/264 [==============================] - 83s 314ms/step - loss: 0.0263 - val_loss: 0.0275
Epoch 5/100
265/264 [==============================] - 84s 317ms/step - loss: 0.0257 - val_loss: 0.0283
Epoch 6/100
265/264 [==============================] - 84s 317ms/step - loss: 0.0251 - val_loss: 0.0271
Epoch 7/100
265/264 [==============================] - 84s 317ms/step - loss: 0.0248 - val_loss: 0.0262
Epoch 8/100
265/264 [==============================] - 84s 317ms/step - loss: 0.0241 - val_loss: 0.0263
Epoch 9/100
265/264 [==============================] - 84s 317ms/step - loss: 0.0238 - val_loss: 0.0251
Epoch 10/100
265/264 [==============================] - 84s 317m